In [14]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
import keras
from keras import models
from keras import layers


In [156]:
# reading dataset from csv

data = pd.read_csv('corr_data.csv')
data.head()

,Unnamed: 0,chroma_stft,spectral_bandwidth,rmse,zero_crossing_rate,rolloff,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc9,mfcc11,mfcc15,mfcc16,mfcc17,mfcc18,label
0,0,0.377688,1972.202261,0.131890,0.058236,3470.404015,115.346103,9.969873,54.051329,7.616936,28.314299,-9.058338,-15.045016,-5.689818,-0.748612,5.065468,-5.084879,-2.795500,blues
1,1,0.336454,2310.305516,0.112245,0.075345,4849.495676,93.629480,6.343454,71.511946,-8.107514,36.207253,-9.776674,-21.115800,-22.199284,-12.401641,9.624600,-5.014185,-3.704146,blues
2,2,0.369973,1772.854380,0.187148,0.057147,2754.525010,139.977099,-1.269146,39.323654,-6.351806,19.769089,-11.213042,-16.532666,-1.197161,-1.633326,0.956659,-4.840757,-5.966955,blues
3,3,0.429511,2290.401739,0.148366,0.048052,4551.155813,89.791041,12.979256,80.691273,3.660787,39.741976,-5.403830,-13.160739,-20.552354,-17.545474,4.154533,-5.990226,-7.860423,blues
4,4,0.397725,2356.855937,0.134892,0.097657,5108.319111,90.701812,-2.527487,74.053272,-11.747301,39.214671,-13.171819,-17.163641,-21.433580,-16.485552,6.542503,-5.695943,-3.640864,blues


In [157]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [158]:
# normalizing
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

# spliting of dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [159]:
# creating a model
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
history = model.fit(X_train,
                    y_train,
                    epochs=500,
                    batch_size=128)

Epoch 1/500
792/792 [==============================] - 0s 602us/step - loss: 2.1927 - acc: 0.2235
Epoch 2/500
792/792 [==============================] - 0s 38us/step - loss: 1.8778 - acc: 0.4091
Epoch 3/500
792/792 [==============================] - 0s 44us/step - loss: 1.6113 - acc: 0.4596
Epoch 4/500
792/792 [==============================] - 0s 41us/step - loss: 1.3634 - acc: 0.5669
Epoch 5/500
792/792 [==============================] - 0s 48us/step - loss: 1.1306 - acc: 0.7008
Epoch 6/500
792/792 [==============================] - 0s 47us/step - loss: 0.9243 - acc: 0.7336
Epoch 7/500
792/792 [==============================] - 0s 42us/step - loss: 0.7535 - acc: 0.7879
Epoch 8/500
792/792 [==============================] - 0s 39us/step - loss: 0.6316 - acc: 0.8081
Epoch 9/500
792/792 [==============================] - 0s 40us/step - loss: 0.5406 - acc: 0.8333
Epoch 10/500
792/792 [==============================] - 0s 35us/step - loss: 0.4704 - acc: 0.8497
Epoch 11/500
792/792 [======

792/792 [==============================] - 0s 47us/step - loss: 0.0067 - acc: 1.0000
Epoch 85/500
792/792 [==============================] - 0s 42us/step - loss: 0.0068 - acc: 1.0000
Epoch 86/500
792/792 [==============================] - 0s 51us/step - loss: 0.0077 - acc: 1.0000
Epoch 87/500
792/792 [==============================] - 0s 40us/step - loss: 0.0073 - acc: 1.0000
Epoch 88/500
792/792 [==============================] - 0s 34us/step - loss: 0.0066 - acc: 1.0000
Epoch 89/500
792/792 [==============================] - 0s 36us/step - loss: 0.0058 - acc: 1.0000
Epoch 90/500
792/792 [==============================] - 0s 42us/step - loss: 0.0057 - acc: 1.0000
Epoch 91/500
792/792 [==============================] - 0s 42us/step - loss: 0.0055 - acc: 1.0000
Epoch 92/500
792/792 [==============================] - 0s 46us/step - loss: 0.0053 - acc: 1.0000
Epoch 93/500
792/792 [==============================] - 0s 41us/step - loss: 0.0049 - acc: 1.0000
Epoch 94/500
792/792 [===========

792/792 [==============================] - 0s 39us/step - loss: 0.0011 - acc: 1.0000
Epoch 168/500
792/792 [==============================] - 0s 40us/step - loss: 0.0010 - acc: 1.0000
Epoch 169/500
792/792 [==============================] - 0s 42us/step - loss: 0.0010 - acc: 1.0000
Epoch 170/500
792/792 [==============================] - 0s 36us/step - loss: 0.0010 - acc: 1.0000
Epoch 171/500
792/792 [==============================] - 0s 38us/step - loss: 9.9785e-04 - acc: 1.0000
Epoch 172/500
792/792 [==============================] - 0s 39us/step - loss: 9.7745e-04 - acc: 1.0000
Epoch 173/500
792/792 [==============================] - 0s 39us/step - loss: 9.6934e-04 - acc: 1.0000
Epoch 174/500
792/792 [==============================] - 0s 41us/step - loss: 9.5541e-04 - acc: 1.0000
Epoch 175/500
792/792 [==============================] - 0s 45us/step - loss: 9.4137e-04 - acc: 1.0000
Epoch 176/500
792/792 [==============================] - 0s 39us/step - loss: 9.7731e-04 - acc: 1.0000


792/792 [==============================] - 0s 43us/step - loss: 3.9578e-04 - acc: 1.0000
Epoch 247/500
792/792 [==============================] - 0s 53us/step - loss: 3.9133e-04 - acc: 1.0000
Epoch 248/500
792/792 [==============================] - 0s 52us/step - loss: 3.9505e-04 - acc: 1.0000
Epoch 249/500
792/792 [==============================] - 0s 46us/step - loss: 3.8827e-04 - acc: 1.0000
Epoch 250/500
792/792 [==============================] - 0s 41us/step - loss: 3.8485e-04 - acc: 1.0000
Epoch 251/500
792/792 [==============================] - 0s 40us/step - loss: 3.8442e-04 - acc: 1.0000
Epoch 252/500
792/792 [==============================] - 0s 40us/step - loss: 3.7563e-04 - acc: 1.0000
Epoch 253/500
792/792 [==============================] - 0s 36us/step - loss: 3.7007e-04 - acc: 1.0000
Epoch 254/500
792/792 [==============================] - 0s 54us/step - loss: 3.6772e-04 - acc: 1.0000
Epoch 255/500
792/792 [==============================] - 0s 40us/step - loss: 3.6222e-0

792/792 [==============================] - 0s 46us/step - loss: 1.9289e-04 - acc: 1.0000
Epoch 326/500
792/792 [==============================] - 0s 39us/step - loss: 1.9256e-04 - acc: 1.0000
Epoch 327/500
792/792 [==============================] - 0s 47us/step - loss: 1.8964e-04 - acc: 1.0000
Epoch 328/500
792/792 [==============================] - 0s 39us/step - loss: 1.8829e-04 - acc: 1.0000
Epoch 329/500
792/792 [==============================] - 0s 44us/step - loss: 1.8662e-04 - acc: 1.0000
Epoch 330/500
792/792 [==============================] - 0s 40us/step - loss: 1.8500e-04 - acc: 1.0000
Epoch 331/500
792/792 [==============================] - 0s 39us/step - loss: 1.8447e-04 - acc: 1.0000
Epoch 332/500
792/792 [==============================] - 0s 42us/step - loss: 1.8303e-04 - acc: 1.0000
Epoch 333/500
792/792 [==============================] - 0s 37us/step - loss: 1.8155e-04 - acc: 1.0000
Epoch 334/500
792/792 [==============================] - 0s 41us/step - loss: 1.7983e-0

792/792 [==============================] - 0s 35us/step - loss: 1.0837e-04 - acc: 1.0000
Epoch 405/500
792/792 [==============================] - 0s 39us/step - loss: 1.0777e-04 - acc: 1.0000
Epoch 406/500
792/792 [==============================] - 0s 41us/step - loss: 1.0714e-04 - acc: 1.0000
Epoch 407/500
792/792 [==============================] - 0s 42us/step - loss: 1.0568e-04 - acc: 1.0000
Epoch 408/500
792/792 [==============================] - 0s 36us/step - loss: 1.0634e-04 - acc: 1.0000
Epoch 409/500
792/792 [==============================] - 0s 40us/step - loss: 1.0503e-04 - acc: 1.0000
Epoch 410/500
792/792 [==============================] - 0s 41us/step - loss: 1.0656e-04 - acc: 1.0000
Epoch 411/500
792/792 [==============================] - 0s 42us/step - loss: 1.0403e-04 - acc: 1.0000
Epoch 412/500
792/792 [==============================] - 0s 48us/step - loss: 1.0305e-04 - acc: 1.0000
Epoch 413/500
792/792 [==============================] - 0s 44us/step - loss: 1.0238e-0

792/792 [==============================] - 0s 41us/step - loss: 6.5534e-05 - acc: 1.0000
Epoch 484/500
792/792 [==============================] - 0s 37us/step - loss: 6.5065e-05 - acc: 1.0000
Epoch 485/500
792/792 [==============================] - 0s 43us/step - loss: 6.4695e-05 - acc: 1.0000
Epoch 486/500
792/792 [==============================] - 0s 39us/step - loss: 6.4285e-05 - acc: 1.0000
Epoch 487/500
792/792 [==============================] - 0s 42us/step - loss: 6.4013e-05 - acc: 1.0000
Epoch 488/500
792/792 [==============================] - 0s 40us/step - loss: 6.3559e-05 - acc: 1.0000
Epoch 489/500
792/792 [==============================] - 0s 41us/step - loss: 6.3305e-05 - acc: 1.0000
Epoch 490/500
792/792 [==============================] - 0s 41us/step - loss: 6.2825e-05 - acc: 1.0000
Epoch 491/500
792/792 [==============================] - 0s 49us/step - loss: 6.2482e-05 - acc: 1.0000
Epoch 492/500
792/792 [==============================] - 0s 37us/step - loss: 6.2225e-0

In [160]:
# calculate accuracy
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

198/198 [==============================] - 0s 590us/step
test_acc:  0.8686868680848016


In [26]:
def predict_class(prediction):
    if prediction == 0:
        return "blues"
    elif prediction == 1:
        return "classical"
    elif prediction == 2:
        return "country"
    elif prediction == 3:
        return "disco"
    elif prediction == 4:
        return "hiphop"
    elif prediction == 5:
        return "jazz"
    elif prediction == 6:
        return "metal"
    elif prediction == 7:
        return "pop"
    elif prediction == 8:
        return "raggae"
    else:
        return "rock"
    

In [146]:
# reading dataset from csv

test_data = pd.read_csv('check_corr_data.csv')
test_data.head()

,Unnamed: 0,spectral_bandwidth,zero_crossing_rate,rolloff,mfcc2,mfcc7,mfcc8,mfcc11,label
0,0,1703.281814,0.074769,3174.215635,136.238846,-1.766903,3.008138,-8.105226,blues
1,1,924.252432,0.055279,1379.887909,162.686022,-7.724746,-13.298053,-11.075872,classical
2,2,2518.903088,0.119726,5293.279997,84.896852,-8.578657,2.398183,-9.692951,disco
3,3,2594.015040,0.095023,5419.113156,92.614165,-3.739453,5.162560,-1.106493,disco
4,4,1977.370661,0.062340,3365.970909,124.441260,-3.133675,8.096397,-9.473353,hiphop


In [147]:
y_test = test_data["label"]
del test_data["label"]
#del test_data["Unnamed: 0"]
X_test = test_data
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

In [148]:
encoder = LabelEncoder()
y_test = encoder.fit_transform(y_test)

In [149]:
print(X_test)

[[-1.54919334 -0.78404263 -0.65513854 -0.76137595  1.04013245  0.32282157
  -0.33286996 -0.74926898]
 [-1.161895   -2.21072731 -1.08014886 -1.93155399  1.7956153  -1.39940206
  -2.51712597 -1.39050581]
 [-0.77459667  0.70965505  0.32527823  0.62058039 -0.42648907 -1.64624079
  -0.41457507 -1.09199174]
 [-0.38729833  0.84721221 -0.21343233  0.70264299 -0.20603839 -0.24738029
  -0.04427958  0.7614612 ]
 [ 0.         -0.28208677 -0.92618307 -0.63632198  0.7031258  -0.07226904
   0.34871542 -1.04458963]
 [ 0.38729833 -0.56617868 -0.71947358 -0.79407756  0.39204652  0.5223968
   0.54975001  0.8473204 ]
 [ 0.77459667  0.48440144  0.75349812  0.61462481 -0.87038596  1.22579208
   0.89696039  0.33505514]
 [ 1.161895    0.99738592  0.22209392  1.02496924 -0.97288175  1.52688069
   0.70263571  0.95281679]
 [ 1.54919334  0.80438076  2.2935061   1.16051205 -1.45512489 -0.23259894
   0.81078904  1.37970262]]


In [150]:
y_test

array([0, 1, 2, 2, 3, 3, 4, 4, 4])

In [151]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

9/9 [==============================] - 0s 123us/step
test_acc:  0.3333333432674408


In [152]:
val = model.predict(X_test)

In [153]:
val

array([[8.85599554e-01, 1.14398874e-01, 1.62505114e-06, 1.56344206e-19,
        5.44663956e-32, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [4.52734974e-18, 9.99998808e-01, 1.16616479e-06, 9.23866975e-17,
        4.50262519e-22, 6.47308425e-28, 1.71554303e-36, 2.39918655e-35,
        0.00000000e+00, 0.00000000e+00],
       [4.32837304e-08, 3.83974103e-11, 9.78318453e-01, 2.16815434e-02,
        4.70273844e-14, 2.07287907e-22, 1.78669628e-28, 9.06751175e-26,
        0.00000000e+00, 0.00000000e+00],
       [1.02934073e-22, 1.22909060e-16, 1.70764054e-07, 9.98981059e-01,
        1.01868855e-03, 2.75111930e-11, 6.10332174e-15, 1.26471527e-15,
        1.07884797e-35, 0.00000000e+00],
       [2.40025640e-28, 3.88419324e-18, 6.60813808e-12, 2.26535306e-07,
        3.06949689e-04, 9.99689579e-01, 3.20174354e-06, 6.10971984e-09,
        6.03481470e-22, 5.56055848e-34],
       [1.24237535e-29, 3.06645521e-19, 2.01885609e-18, 1.60514716e-16,
   

In [154]:
classes = model.predict_classes(X_test)
print(classes)

[0 1 2 3 5 6 6 8 9]


In [31]:
genre_list = test_data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 1 1]


In [33]:
# normalizing
scaler = StandardScaler()
X1 = scaler.fit_transform(np.array(test_data.iloc[:, :-1], dtype = float))

# spliting of dataset into train and test dataset
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.99)

In [61]:
print(X1_test)

[[-1.22474487 -1.40099184 -1.16091394 -1.40537646  1.38237332 -1.34809054
  -1.41178404 -0.87562618]
 [ 0.          0.86757794 -0.11897788  0.8393914  -0.43276259  0.30392514
   0.63413298 -0.52393485]
 [ 1.22474487  0.5334139   1.27989182  0.56598506 -0.94961073  1.04416541
   0.77765107  1.39956103]]


In [54]:
correct = 0
wrong = 0
#predictions = model.predict(test_data)
predictions = model.predict(my_data)
for index in range(len(predictions)): 
    prediction = np.argmax(predictions[index])
    print(prediction)
    audio_name = predict_class(prediction)
    print(predict_class(y1_test[index]))
    print(audio_name)
    if predict_class(y1_test[index]) == audio_name:
        correct = correct+1
    else:
        wrong = wrong+1
        
    print("***************************************************")
print("correct",correct)
print("wrong",wrong)

ValueError: Error when checking input: expected dense_9_input to have shape (8,) but got array with shape (1,)

In [11]:
type(y_train)

numpy.ndarray

In [12]:
unique, counts = np.unique(y_train, return_counts=True)


In [13]:
print(np.asarray((unique, counts)).T)

[[ 0 73]
 [ 1 87]
 [ 2 85]
 [ 3 79]
 [ 4 78]
 [ 5 77]
 [ 6 73]
 [ 7 81]
 [ 8 79]
 [ 9 80]]
